In [1]:
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import torch.utils.data
import cv2
import torchvision.models.segmentation
import torch
import os
import patchify
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image as skimg
import imgaug as ia
import imgaug.augmenters as iaa
from torchvision.models import resnet50, ResNet50_Weights, ResNet101_Weights
from torchvision.models.detection.mask_rcnn import MaskRCNN_ResNet50_FPN_V2_Weights,MaskRCNN_ResNet50_FPN_Weights
import torch.optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.insert(1, '/home/prakharug/AFO')
sys.path.insert(1, '/home/prakharug/AFO/pycoco')
from pycoco.engine import train_one_epoch, evaluate
import miou_eval as mi

In [2]:
def combine_dims(a, start=0, count=2):
    """ Reshapes numpy array a by combining count dimensions, 
        starting at dimension index start """
    s = a.shape
    return np.reshape(a, s[:start] + (-1,) + s[start+count:])


def collate_fn(batch):
    return tuple(zip(*batch))

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 
print(device)

cuda


In [4]:
def evalm(model,fintestloader):
    model.eval()
    iou = 0
    cnt = 0
    for i, data in enumerate(fintestloader, 0):
        images, targets = data
        #print(type(targets))
        if(images[0]=="Problem"):
            continue
        #print(type(images[0]))
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        #print(model(images)[0]["masks"].squeeze().shape,targets[0]["masks"].shape)
        tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
        #print(tiou/tcnt)
        #print("Txnt: ",tcnt)
        del images,targets
        iou += tiou
        cnt += tcnt
        #print(iou/cnt)
    print("mIoU on test is:",iou.item()/cnt)
    model.train()
    return iou.item()/cnt
    

In [5]:
def evalmt(model,fintestloader):
    model.eval()
    iou = 0
    cnt = 0
    for i, data in enumerate(fintestloader, 0):
        images, targets = data
        seq2 = iaa.Sequential([
        iaa.Fliplr(0.5), # horizontal flips
        # Strengthen or weaken the contrast in each image.
        iaa.LinearContrast((0.75, 1.5))],
        random_order=True) 
        #print(type(targets))
        if(images[0]=="Problem"):
            continue
        images1 = seq2(images=images)
        images2 = seq2(images=images)
        images3 = seq2(images=images)
        #print(type(images[0]))
        images = list(image.to(device) for image in images)
        images1 = list(image.to(device) for image in images1)
        images2 = list(image.to(device) for image in images2)
        images3 = list(image.to(device) for image in images3)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        #print(model(images)[0]["masks"].squeeze().shape,targets[0]["masks"].shape)
        tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
        tiou1,tcnt1 = mi.miou_eval(targets[0]["masks"],(model(images1)[0]["masks"].squeeze(1)>0.5))
        tiou2,tcnt2 = mi.miou_eval(targets[0]["masks"],(model(images2)[0]["masks"].squeeze(1)>0.5))
        tiou3,tcnt3 = mi.miou_eval(targets[0]["masks"],(model(images3)[0]["masks"].squeeze(1)>0.5))
        #print(tiou/tcnt)
        #print("Txnt: ",tcnt)
        del images,targets
        iou += tiou
        cnt += tcnt
        #print(iou/cnt)
    print("mIoU on test is:",iou.item()/cnt)
    model.train()
    return iou.item()/cnt
    

In [6]:
class MMCellDataset(Dataset):
    def __init__(self,root_dir, tester = False, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.img = []
        self.mean = [0.485, 0.456, 0.406]
        self.dev = [0.229, 0.224, 0.225]
        self.normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        self.nml = []
        self.tester = tester
        self.datano =  root_dir.split('/')[1]
        print(self.datano)
        for im_name in os.listdir(self.root_dir):
            self.nml.append(im_name)
            # tmplist = []
            # lent = len(im_name[:-4])
            # print(im_name,im_name[:lent])
            # imgtmp = cv2.imread(self.root_dir+im_name,1).transpose(2,0,1)
            # tmplist.append(imgtmp[0])
            # tmplist.append(imgtmp[1])
            # tmplist.append(imgtmp[2])
            # for gt_name in os.listdir("../"+self.datano+"/ground_truths"):
            #     if gt_name[0:0+lent]==im_name[:lent] and gt_name[0+lent]=="_":
            #         mask = np.array(cv2.imread("../"+self.datano+"/ground_truths/"+gt_name,0))
            #         tmplist.append(mask)
            # tmplist = np.array(tmplist)
            # #print(tmplist.shape)
            # self.img.append(tmplist)   
    
    def __len__(self):
        return len([name for name in os.listdir(self.root_dir)])

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        im_name = self.nml[idx]
        tmplist = []
        lent = len(im_name[:-4])
        #print(im_name,im_name[:lent])
        imgtmp = cv2.imread(self.root_dir+im_name,1).transpose(2,0,1)
        # scale_percent = 100 # percent of original size
        # width = int(imgtmp.shape[1] * scale_percent / 100)
        # height = int(imgtmp.shape[0] * scale_percent / 100)
        # dim = (width, height)
        # imgtmp = cv2.resize(imgtmp, dim, interpolation = cv2.INTER_AREA)
        #cv2.imwrite("./result1.png",imgtmp.transpose(1,2,0))
        tmplist.append(imgtmp[0])
        tmplist.append(imgtmp[1])
        tmplist.append(imgtmp[2])
        masker = np.zeros(imgtmp[0].shape)
        for gt_name in os.listdir("../"+self.datano+"/ground_truths"):
            if gt_name[0:0+lent]==im_name[:lent] and gt_name[0+lent]=="_":
                mask = np.array(cv2.imread("../"+self.datano+"/ground_truths/"+gt_name,0))
                masker+=mask*5
                tmplist.append(mask)
        tmplist = np.array(tmplist)
        #print(masker.shape,type(masker))
        masker = np.minimum(masker,255).astype(np.uint8)
        #print(masker)
        #print(cv2.imwrite("./result.png",np.array([masker]).transpose(1,2,0)))
        #print("Tmplist: ",tmplist.shape)
        patch_stack = tmplist
        patch_stack = np.array(patch_stack)
        k_w = random.randint(700,patch_stack.shape[2])
        k_h = random.randint(700,patch_stack.shape[1])
        # k_w = patch_stack.shape[2]
        # k_h = patch_stack.shape[1]
        o_w = random.randint(0,patch_stack.shape[2]-k_w)
        o_h = random.randint(0,patch_stack.shape[1]-k_h)
        patch_img = patch_stack[0:3,o_h:o_h+k_h,o_w:o_w+k_w]
        patch_img = cv2.cvtColor(patch_img.transpose(1,2,0), cv2.COLOR_BGR2LAB ).transpose(2,0,1)
        instances = patch_stack[3:,o_h:o_h+k_h,o_w:o_w+k_w]
        instances = instances.transpose(1,2,0)
        data = {}
        masks = []
        boxes = []
        area = []
        t=0
        for a in range(instances.shape[2]):
            dispim = instances[:,:,a]
            if np.all(dispim == 0):
                continue
            x,y,w,h = cv2.boundingRect(dispim)
            boxes.append([x, y, x+w, y+h])
            area.append(torch.tensor(h*w))
            masks.append(dispim/255)
            t=1
        if t==0:
            #print("Abort")
            if self.tester:
                return "Problem","Hao gai"
            else:
                #print("Abort Abort ")
                return self.__getitem__((idx+1)%len(self.nml))
        masks = np.array(masks)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        area = torch.as_tensor(area, dtype=torch.float32)
        img = torch.as_tensor(patch_img, dtype=torch.float32)
        img = img/255
        img = self.normalize(img)
        data["boxes"] =  boxes
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        data["iscrowd"] = iscrowd
        data["labels"] =  torch.ones((boxes.shape[0],), dtype=torch.int64)   # there is only one class
        data["masks"] = masks
        data["area"] = area
        data["image_id"] = torch.tensor(idx)
        return img,data
# train = MMCellDataset("../bigW/images/")
# train1 = MMCellDataset("../dataset1/test/")
# train1 = MMCellDataset("../dataset1/images/")
train = MMCellDataset("../dataset11/images/")
#test = MMCellDataset("../dataset12/test/")
fintest = MMCellDataset("../test/images/",True)
# train = torch.utils.data.ConcatDataset([train, train1])
#print(len(test))
trainloader = DataLoader(train, batch_size=4, shuffle=True,collate_fn = collate_fn,num_workers=10)
#testloader = DataLoader(test, batch_size=1, shuffle=True,collate_fn = collate_fn,num_workers=10)
fintestloader = DataLoader(fintest, batch_size=1, shuffle=False,collate_fn = collate_fn,num_workers=10)

dataset11
test


In [8]:
class alchemy(nn.Module):
    def __init__(self,**kwargs) -> None:
        super(alchemy, self).__init__()
        self.vis = False
        self.maskrcnn = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT,trainable_backbone_layers=4,**kwargs)
        in_features = self.maskrcnn.roi_heads.box_predictor.cls_score.in_features  # get number of input features for the classifier
        self.maskrcnn.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes=2)  # replace the pre-trained head with a new one

    def forward(self, images, targets=None):
        ret = self.maskrcnn(images,targets)
        return ret

In [ ]:
from  torchvision.models.detection.backbone_utils import _resnet_fpn_extractor
class alchemy2(nn.Module):
    def __init__(self,**kwargs) -> None:
        super(alchemy2, self).__init__()
        self.vis = False
        self.maskrcnn = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT,trainable_backbone_layers=4,**kwargs)
        self.backbone = torchvision.models.resnet101(weights=ResNet101_Weights.DEFAULT )
        self.backbone = _resnet_fpn_extractor(self.backbone, 4, norm_layer=nn.BatchNorm2d)
        self.maskrcnn.backbone = self.backbone
        in_features = self.maskrcnn.roi_heads.box_predictor.cls_score.in_features  # get number of input features for the classifier
        self.maskrcnn.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes=2)  # replace the pre-trained head with a new one

    def forward(self, images, targets=None):
        ret = self.maskrcnn(images,targets)
        return ret

In [ ]:
model = alchemy()
model.to(device)
#model.load_state_dict(torch.load("alchemy_0_9146_d4.torch"))
#model.load_state_dict(torch.load("alchemy_0_9116.torch"))
optimizer = torch.optim.AdamW(model.parameters(), lr=8e-5)
model.train()
lmbda = lambda epoch: 0.1
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)
# del model

In [ ]:
for epoch in range(30): 

    model.train()

    for i, data in enumerate(trainloader, 0):
        images, targets = data
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        #print(targets[0]["boxes"].shape)
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        del images,targets
        print(i,'loss:', losses.item())
    
    torch.save(model.state_dict(), "alchemy_"+str(epoch)+".torch")
    #scheduler.step()


    model.eval()
    #evaluate(model, testloader, device=device)
    iou = 0
    cnt = 0
    
    for i, data in enumerate(testloader, 0):
        images, targets = data
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        #print(type(model(images)[0]["masks"]))
        tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
        del images,targets
        iou += tiou
        cnt += tcnt
    print("mIoU on val is:",iou.item()/cnt)

    if epoch%1==0:
            model.eval()
            iou = 0
            cnt = 0
            for i, data in enumerate(fintestloader, 0):
                images, targets = data
                #print(type(targets))
                if(images[0]=="Problem"):
                    continue
                #print(type(images[0]))
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                #print(model(images)[0]["masks"].squeeze().shape,targets[0]["masks"].shape)
                tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
                #print("Txnt: ",tcnt)
                del images,targets
                iou += tiou
                cnt += tcnt
                #print(iou/cnt)
            print("mIoU on test is:",iou.item()/cnt)


print('Finished Training')

In [ ]:
model1.eval()
iou = 0
cnt = 0
for i, data in enumerate(fintestloader, 0):
    images, targets = data
    #print(type(targets))
    if(images[0]=="Problem"):
        continue
    #print(type(images[0]))
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    #print(model(images)[0]["masks"].squeeze().shape,targets[0]["masks"].shape)
    tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model1(images)[0]["masks"].squeeze(1)>0.5))
    #print("Txnt: ",tcnt)
    del images,targets
    iou += tiou
    cnt += tcnt
#print(iou/cnt)
print("mIoU on test is:",iou.item()/cnt)

In [ ]:
model = alchemy(box_nms_thresh=0.5)
model.to('cuda')
model.load_state_dict(torch.load("./alchemy_0_0.9317_d20.torch"))
model.eval()
#model1 = model
for im_name in os.listdir("./submission/x/"):
    imgtmp = cv2.imread("./submission/x/"+im_name,1)
    imgtmp = cv2.cvtColor(imgtmp, cv2.COLOR_BGR2LAB ).transpose(2,0,1)
    imgtmp = torch.as_tensor(imgtmp, dtype=torch.float32)
    imgtmp = imgtmp/255
    imgtmp = imgtmp.to(device)
    output = model([imgtmp])
    prefix = im_name[0:-4]
    ref = 0
    for i in range(output[0]['masks'].shape[0]):
        print(output[0]['scores'][i].item())
        it = torch.clone(output[0]['masks'][i])
        it = it.detach().to('cpu').numpy()
        it = it.squeeze(0)
        it = (it>=0.5).astype(float)
        it = it*255
        cv2.imwrite("./submission/y/"+prefix+"_"+str(i)+".bmp",it)

In [ ]:
! make sub

In [ ]:
test = set()
for im_name in os.listdir('./submission/y/'):
    if(im_name[4]!='.'):
        test.add(im_name[0:4])

print(len(test))
print(test)

test = set()
for im_name in os.listdir('./submission/x/'):
    test.add(im_name[0:4])

print(len(test))
print(test)


In [ ]:
model = alchemy()
model.to(device)
model.load_state_dict(torch.load("./alchemy_7_0.9183.torch"))
torch.save(model.maskrcnn.backbone.state_dict(),"./resnet_0.9183.pth")

In [9]:
min_val = 0.91
for iv in range(20):
    model = alchemy()
    model.to(device)
    # model.load_state_dict(torch.load("./alchemyi_2_0.9283.torch"))
    # model.maskrcnn.backbone.load_state_dict(torch.load("./new_resnet2.0.torch"))
    #model.load_state_dict(torch.load("alchemy_0_9116.torch"))
    # lr=0.1
    # if(iv>=10==0):
    #     lr=6e-5
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01,momentum=0.9,weight_decay=0.00004)
    #optimizer = torch.optim.AdamW(model.parameters(), lr=8e-5)
    model.train()
    lmbda = lambda epoch: 1
    scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,200)
    for epoch in range(4): 
        model.train()
        lrs = 0
        for i, data in enumerate(trainloader, 0):
            lrs+=1
            images, targets = data
            optimizer.zero_grad()
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            #if losses.item()>=0.2:
            losses.backward()
            optimizer.step()
            
            #print("lr: ",optimizer.param_groups[0]['lr'])
            print(i,'loss:', losses.item())
            del images,targets,losses
            if lrs==100:
                lrs=0
                #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 200)
                # for param_group in optimizer.param_groups:
                #     param_group['lr'] = lr*0.6
                #     lr = lr * 0.6
                val = evalm(model,fintestloader)
                if(val>min_val):
                    print("saved ",str(i)," with mIoU:",val)
                    min_val=val
                    torch.save(model.state_dict(), "test_"+str(iv)+"_"+str(val)[0:6]+"_d20.torch")
        #val = evalm(model,fintestloader)
        val = evalm(model,fintestloader)
        if(val>min_val):
            print("saved ",str(i)," with mIoU:",val)
            min_val=val
            torch.save(model.state_dict(), "test_"+str(iv)+"_"+str(val)[0:6]+"_d20.torch")
        #torch.save(model.state_dict(), "alchemy_"+str(iv)+".torch")
    #scheduler.step()  
        
    del model

print('Finished Training')

0 loss: 2.889272928237915
1 loss: 2.142735481262207
2 loss: 1.9186341762542725
3 loss: 1.6510980129241943
4 loss: 1.8573025465011597
5 loss: 1.1198457479476929
6 loss: 1.140470027923584
7 loss: 1.0378395318984985
8 loss: 0.967262327671051
9 loss: 0.7411631345748901
10 loss: 0.8660396933555603
11 loss: 0.7378925681114197
12 loss: 0.6820128560066223
13 loss: 0.7641640901565552
14 loss: 0.5853605270385742
15 loss: 0.5284894704818726
16 loss: 0.5999065041542053
17 loss: 0.49230310320854187
18 loss: 0.535049319267273
19 loss: 0.6601464748382568
20 loss: 0.6186835169792175
21 loss: 0.5038853883743286
22 loss: 0.46592313051223755
23 loss: 0.765199601650238
24 loss: 0.7244481444358826
25 loss: 0.4530074894428253
26 loss: 0.41767486929893494
27 loss: 0.47089797258377075
28 loss: 0.4143081605434418
29 loss: 0.3950876295566559
30 loss: 0.4874890446662903
31 loss: 0.36276811361312866
32 loss: 0.41996967792510986
33 loss: 0.379890501499176
34 loss: 0.4272792935371399
35 loss: 1.242864966392517
36 l

In [ ]:
torch.cuda.memory_summary()

In [ ]:
min_val=0.92
model = alchemy()
model.to(device)
#model.load_state_dict(torch.load("alchemy_0_9146_d4.torch"))
#model.load_state_dict(torch.load("alchemy_0_9116.torch"))
lr=0.1
optimizer = torch.optim.SGD(model.parameters(), lr=0.1,weight_decay=0.00004)
#optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
model.train()
lmbda = lambda epoch: 0.5
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,200)
for epoch in range(10): 
    model.train()
    lrs = 0
    for i, data in enumerate(trainloader, 0):
        lrs+=1
        images, targets = data
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        if losses.item()>=0.20:
            losses.backward()
        optimizer.step()
        
        #print("lr: ",optimizer.param_groups[0]['lr'])
        print(i,'loss:', losses.item())
        del images,targets, losses
        if lrs==30:
            lrs=0
            #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 200)
            # for param_group in optimizer.param_groups:
            #     param_group['lr'] = lr*0.6
            #     lr = lr * 0.6
            val = evalm(model,fintestloader)
            if(val>min_val):
                print("saved ",str(i)," with mIoU:",val)
                min_val=val
                torch.save(model.state_dict(), "alchemyi_"+str(epoch)+"_"+str(val)[0:6]+".torch")
    #val = evalm(model,fintestloader)
    scheduler.step()  
torch.save(model.state_dict(), "alchemy_4.torch")


print('Finished Training')

In [ ]:
from collections import OrderedDict


model = alchemy()
model.to(device)
model.load_state_dict(torch.load("./alchemy_7_0.9183.torch"))
# trad = torch.load("./new_resnet1.torch")
# newt = OrderedDict()
# for i in trad.keys():
#     if(i[0:8]=="backbone"):
#         newt["body."+i[9:]] = trad[i]
#     else:
#         newt[i] = trad[i]
model.maskrcnn.backbone.load_state_dict(torch.load("./new_resnet1.5.torch"),strict=True)
# torch.save(model.maskrcnn.backbone.state_dict(),"./resnetfpnnew.torch")
evalm(model,fintestloader)

In [ ]:
out = fintest[3]

In [ ]:
model = alchemy()
model.load_state_dict(torch.load("./alchemy_0_0.9317_d20.torch"))
model.to(device)
model.eval()
res = model(out)
print(res)

In [ ]:
model = alchemy()
model.load_state_dict(torch.load("alchemyi_0_0.9248.torch"))

torch.save(model.maskrcnn.backbone.state_dict(),"resenter2.1.torch")